## Probabilistic PCA
Reference: chapter 12, Bishop book

In [235]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from sklearn.model_selection import train_test_split
import os, pickle
import numpy as np
from numpy import log, sum, exp, prod
from numpy.linalg import det
from numpy.random import beta, binomial, dirichlet, uniform, gamma, seed, multinomial, gumbel, rand, multivariate_normal, normal
from scipy.stats import wishart #, norm, randint, bernoulli, beta, multinomial, gamma, dirichlet, uniform
from scipy.special import digamma
from imp import reload
from copy import deepcopy
#import seaborn as sns
import pandas as pd


In [293]:
N = 10**4
D = 20
K = 5    # latent dimension M in book
MCsim = 30
burnin = 10

### Generate data: 

In [318]:
# DGP:

#np.random.seed(0)

sigma2_true = 1.5
z = normal(loc=0, scale=1, size = (K,1)); z.shape
mu_true = uniform(size = (D,1)); mu.shape
W_true = normal(loc=0, scale=1, size = (D,K))
#W_true = uniform(0,5, size = (D,K)); W_true.shape
mean_true = (W_true @ z + mu_true).squeeze()

X = multivariate_normal(mean = mean_true, cov = sigma2_true * np.eye(D), size = N); X.shape
#X = normal(loc=0, scale=1, size = (N,D)); X.shape

(10000, 20)

In [310]:
# Standardize columns:
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
X.shape

#https://github.com/polydefk/Probabilistic-PCA/blob/master/PPCA.py

(10000, 20)

In [319]:
#-------------
# Initialize:
#-------------

np.random.seed(0)

#W = np.empty((D,K,MCsim)) 
sigma2 = np.empty((MCsim))           

i = 0
#W[:,:,i] = normal(loc=0, scale=1, size = (D,K))
W = np.random.rand(D, K)
#W[:,:,i] = np.random.rand(D, K)
sigma2[i] = gamma(1)

x_mean = np.mean(X, axis=0)       # estimator of mu
X_mean = np.tile(x_mean, (N,1))

for i in range(1,MCsim):

    if i % 10 == 0: print("Iter: ",i)

    #--------
    # E-Step
    #--------
    #M = (W[:,:,i-1].T) @ W[:,:,i-1] + (sigma2[0,i-1] * np.eye(K))
    M = W.T @ W + (sigma2[i-1] * np.eye(K))
    M1 = np.linalg.inv(M) 

    Ez = M1 @ W.T @ (X - X_mean).T       # 12.54
    #Ez = M1 @ (W[:,:,i-1].T) @ (X - X_mean).T       # 12.54
    
    #
    #Minv = np.linalg.inv(W.T.dot(W) + sigma2[i-1] * np.identity(K))
    #Xn = Minv.dot(np.transpose(W)).dot((X - mean).T)
    #XnXn = sigma2[i-1] * Minv + Xn.dot(np.transpose(Xn))

    # Maximization Step
    #W = (np.transpose(X - mean).dot(np.transpose(Xn))).dot(np.linalg.inv(XnXn))
    
    z_moment2 = np.empty((N,K,K)) ; #x_cent = (X - X_mean).T ; cross_pr = np.empty((N,D,K))
    for n in range(N) :    
        z_moment2[n,:,:] = sigma2[i-1] * M1 + np.dot(Ez[:,n].reshape(K,1),Ez[:,n].reshape(1,K))     
    #    cross_pr[n,:,:] = np.dot(x_cent[:,n].reshape(D,1) , Ez[:,n].reshape(1,K))

    Ezz = z_moment2.sum(axis=0)       # 12.55
    #x_Ez = cross_pr.sum(axis=0)

    #----------
    # M-step
    #----------
    W = (X - X_mean).T @ Ez.T @ np.linalg.inv(Ezz)     # 12.56

    # Do summing in 12.57 term-wise:
    first_term = np.sum((X - X_mean)**2)

    # 2nd term...
    second_term = -2*Ez.T.sum(axis=0) @ W.T @ (X - X_mean).sum(axis=0)
    #second_term = -2*Ez.T. @ W.T @ (X - X_mean)

    third_term = np.trace(z_moment2[i,:,:] @ W.T @ W)
    #third_term = np.trace(z_moment2[i,:,:] @ W[:,:,i].T @ W[:,:,i])

    sigma2[i] = (first_term + second_term + third_term)/(N*D)

    #sigmaNew = (1 / (N * D)) * \
    #                   (np.square(np.linalg.norm(X - mean)) -
    #                    2 * np.trace(np.transpose(Xn).dot(np.transpose(W_avg)).dot((X - mean).T))) + \
    #                   np.trace(XnXn.dot(np.transpose(W_avg).dot(W_avg)))

    #sigmaNew = np.absolute(sigmaNew)

    #W = W_avg
    #sigma2[0,i] = sigmaNew
    
    
print('Completed!')    


Iter:  10
Iter:  20
Completed!


In [320]:
sigma2[MCsim-1]

1.4990099502772156

In [321]:
W_true

array([[-0.57110209,  0.11887645,  1.03214498,  0.53995243, -2.25126119],
       [-0.34361726, -0.06794353,  1.17658723, -1.51501515, -0.89367181],
       [-0.68024363,  0.19420664, -0.9674974 ,  1.06825969,  1.21308761],
       [ 1.1447188 ,  1.67840445,  0.47840488,  0.09173765,  0.04265348],
       [ 1.36425174,  1.09958891, -0.62773444, -0.12931682, -0.89853941],
       [ 1.93064772,  0.19860651, -0.14580725, -0.21911445,  1.76249777],
       [-1.28223947,  0.26982679,  0.8786986 ,  0.00796627,  1.08857292],
       [ 0.0789783 ,  0.80608233,  0.51910492, -0.12728445, -0.84098401],
       [-0.4953062 , -0.13702   ,  0.9874245 , -0.71854968,  1.93803498],
       [ 0.22039131, -0.47509489,  0.23865399,  0.40564244,  0.29965873],
       [ 0.69116504, -1.90583721,  0.28204433, -2.28763993, -0.55147413],
       [ 1.20762067, -1.15179807,  0.1551826 , -0.63200906, -1.81677267],
       [-0.1880149 ,  0.08906671, -1.6053939 , -0.70751559, -2.14910446],
       [ 0.12630375, -0.0436291 ,  1.0

In [322]:
W

array([[ 1.93001112e-02,  1.02814236e-01,  5.83465837e-02,
         1.03643840e-02,  6.80265907e-02],
       [ 3.55173395e-02, -7.64354490e-03,  2.65897091e-02,
         1.86013264e-01, -3.02539488e-02],
       [ 1.10244996e-01, -1.71634454e-02, -2.42323554e-02,
         4.10278494e-02, -6.09283420e-02],
       [-1.04053399e-01, -7.77798758e-02,  9.70354160e-02,
         6.54946222e-02,  8.79490166e-02],
       [ 1.57994918e-01,  1.36227283e-02,  7.03859628e-02,
         2.06372475e-02, -3.13557420e-02],
       [ 1.42582069e-02, -5.90290012e-02,  1.02442102e-01,
        -4.35333291e-03, -4.19910987e-02],
       [-7.21715156e-02,  7.27000393e-02,  1.64790727e-02,
         6.97372715e-02,  6.45639957e-03],
       [ 9.51535307e-03,  2.36069483e-02,  8.33116633e-02,
         2.53278734e-02,  8.74240787e-02],
       [ 7.87962804e-02,  4.50664152e-02,  5.70012609e-02,
        -7.99748382e-02,  8.87316327e-02],
       [ 6.20234930e-02,  3.45429281e-02, -9.28979492e-02,
         1.81452520e-02

In [315]:
from sklearn.decomposition import PCA

pca = PCA(n_components=K)
pca.fit(X)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.05307604 0.05224013 0.0519389  0.05188245 0.05140003]
[103.03012709 102.2155846  101.92045689 101.86505838 101.39036651]


In [316]:
pca.components_.T#.shape

array([[-0.30530365,  0.17671333,  0.2482287 , -0.06171227,  0.23767859],
       [ 0.06645569, -0.12651763,  0.28758671, -0.11996385,  0.09026344],
       [ 0.453762  , -0.21786545,  0.00700152, -0.09077349, -0.34035534],
       [ 0.12987599,  0.15437655,  0.04723429, -0.40817518, -0.14566386],
       [ 0.23035664,  0.05253995, -0.16521437,  0.15467591,  0.25552587],
       [-0.09646226,  0.16884393, -0.29660299, -0.15189512,  0.10958223],
       [ 0.35859568,  0.08054085, -0.13775706,  0.34942608,  0.30394206],
       [ 0.22563083, -0.18492968, -0.22652266, -0.20080433,  0.2792982 ],
       [ 0.01004149,  0.09530504, -0.39811453, -0.02963334,  0.30502499],
       [ 0.06689497,  0.41251811,  0.1399193 ,  0.49835768, -0.03955526],
       [-0.34227757, -0.10541061,  0.0863469 ,  0.13133852, -0.0076148 ],
       [ 0.09164116,  0.44234455,  0.1990096 , -0.25592854,  0.07671289],
       [ 0.10170061,  0.16445864,  0.10932711, -0.00847113,  0.06200025],
       [-0.15107231,  0.20170381, -0.1

In [317]:
#W.mean(axis=2)
W.shape

(20, 5)

In [199]:
np.random.seed(0)

n=5
d=3
k = 2
x = np.random.poisson(5, (n,d))
z = np.random.poisson(5, (n,k))
x.T

array([[9, 5, 4, 3, 3],
       [5, 5, 5, 3, 3],
       [6, 8, 4, 7, 4]])

In [201]:
np.dot(x.T, z)

array([[117, 135],
       [104, 137],
       [133, 188]])

In [211]:
s=0
for i in range(n):
    s += x.T[:,i].reshape(d,1) @ z[i,:].reshape(1,k)

In [212]:
s

array([[117, 135],
       [104, 137],
       [133, 188]])